In [1]:
# Import necessary packages
import pm4py
import pandas as pd
import json
import os
import shutil

from augm_baseline.ml.core.model import AugmentationExperiment
from augm_baseline.ml.core import model
from augm_baseline.ml.pipeline.augmentation_pipeline import run_pipeline

# Generate baseline data

In this notebook we apply the augmentation appraoch *Model-Agnostic Event Log Augmentation for Predictive Process Monitoring* from *M. Käppel* and *S. Jablonski* with the source code available at: https://github.com/mkaep/pbpm-ssl-suite

In [2]:
# Generate log file for training data
for name in ['helpdesk', 'sepsis', 'bpic13c', 'bpic15_1']:

    # Load data
    train_orig = pd.read_csv(f'data\\{name}\\{name}_orig\\train_{name}_orig.csv')
    train_orig['time:timestamp'] = pd.to_datetime(train_orig['time:timestamp'])
    train_orig[['case:concept:name', 'concept:name']] = train_orig[['case:concept:name', 'concept:name']].astype(str)

    # Convert it to an event log and save it
    train_xes = pm4py.convert_to_event_log(train_orig)
    pm4py.write_xes(train_xes, f'data\\{name}\\{name}_orig\\train_{name}_orig_80.xes')

exporting log, completed traces ::   0%|          | 0/2931 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/672 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/951 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/767 [00:00<?, ?it/s]

After generating the event log files we can now load the defined experiments and create the augmented training data based on this baseline approach.

In [3]:
for name in ['helpdesk', 'sepsis', 'bpic13c', 'bpic15_1']:

    # Load experiment from baseline folder
    with open(f'augm_baseline\\experiments\\aug_{name}.json', 'r') as file:
        exp_data = json.load(file)

    # Generate experiment
    experiment = AugmentationExperiment.from_dict(exp_data)

    # Run the augmentation pipeline
    run_pipeline(experiment, verbose=True)

    # Load augmented data
    df = pd.read_csv(f'augm_baseline\\data\\{name}\\3_mixed_True_1.6\\train_3_mixed_True_1.6_augm.csv')
    df = df[['case:concept:name', 'concept:name', 'time:timestamp']]
    
    # Adjust case IDs as in some approaches we need numerical case IDs
    df['case:concept:name'] = df['case:concept:name'].str.replace('_', '', regex = False)

    # Save baseline training data
    os.makedirs(f'data\\{name}\{name}_baseline', exist_ok=True)
    df.to_csv(f'data\\{name}\\{name}_baseline\\train_{name}_baseline.csv', index=False)

    # Load original validation and test data
    df_val = pd.read_csv(f'data\\{name}\\{name}_orig\\val_{name}_orig.csv')
    df_test = pd.read_csv(f'data\\{name}\\{name}_orig\\test_{name}_orig.csv')

    # Generate full dataset with augmented training data
    df_total = pd.concat([df, df_val, df_test]).reset_index(drop=True)

    # Save full dataset
    total_path = f'data\\{name}\\{name}_baseline\\{name}_baseline.csv'
    df_total.to_csv(total_path, index=False)

    # Move original validation and test data to new folder
    for type in ['val', 'test']:
        source = f'data\\{name}\\{name}_orig\\{type}_{name}_orig.csv'
        dest = f'data\\{name}\\{name}_baseline\\{type}_{name}_baseline.csv'

        shutil.copy2(src=source, dst=dest)

 Start creating folder for 1 event logs and 0 approaches
 Create directory for event log helpdesk
 Load event log from data\helpdesk\helpdesk_orig\train_helpdesk_orig_80.xes


parsing log, completed traces ::   0%|          | 0/2931 [00:00<?, ?it/s]

Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Resolve SW anomaly', 'RESOLVED', 'Closed', 'Create SW anomaly', 'INVALID', 'Resolve ticket', 'Wait', 'Schedule intervention', 'Take in charge ticket', 'Assign seriousness', 'Insert ticket', 'VERIFIED']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Resolve SW anomaly', 'RESOLVED', 'Closed', 'Create SW anomaly', 'INVALID', 'Resolve ticket', 'Wait', 'Schedule intervention', 'Take in charge ticket', 'Assign seriousness', 'Insert ticket', 'VERIFIED']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
Starting augmentation
Augmentor could not appli

parsing log, completed traces ::   0%|          | 0/672 [00:00<?, ?it/s]

Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Admission IC', 'Admission NC', 'Release D', 'Release E', 'Release C', 'IV Liquid', 'CRP', 'Release A', 'Return ER', 'Leucocytes', 'Release B', 'ER Triage', 'LacticAcid', 'ER Registration', 'IV Antibiotics', 'ER Sepsis Triage']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Admission IC', 'Admission NC', 'Release D', 'Release E', 'Release C', 'IV Liquid', 'CRP', 'Release A', 'Return ER', 'Leucocytes', 'Release B', 'ER Triage', 'LacticAcid', 'ER Registration', 'IV Antibiotics', 'ER Sepsis Triage']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.

parsing log, completed traces ::   0%|          | 0/951 [00:00<?, ?it/s]

Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Unmatched', 'Completed', 'Queued', 'Accepted']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Unmatched', 'Completed', 'Queued', 'Accepted']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
Starting augmentation
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applie

parsing log, completed traces ::   0%|          | 0/767 [00:00<?, ?it/s]

Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['01_BB_765', '07_OPS_075', '14_VRIJ_011', '01_HOOFD_195_1', '08_AWB45_050', '01_HOOFD_120', '06_VD_060', '11_AH_II_070', '08_AWB45_090_1', '07_OPS_020', '07_OPS_080_1', '08_AWB45_175', '12_AP_UOV_020_1', '01_HOOFD_491a', '01_HOOFD_195', '02_DRZ_010', '10_UOV_045', '01_HOOFD_110_1', '07_OPS_060_1', '08_AWB45_051_2', '12_AP_UOV_010', '10_UOV_120', '04_BPT_040', '11_AH_II_036', '01_HOOFD_090', '02_DRZ_045', '12_AP_UOV_011', '02_DRZ_030_0', '08_AWB45_070_2', '10_UOV_050_4', '01_BB_546', '01_HOOFD_600_2', '01_BB_670_2', '08_AWB45_051_0', '01_HOOFD_080', '08_AWB45_070_3a', '01_BB_760', '07_OPS_010', '01_HOOFD_010', '04_BPT_060', '02_DRZ_010_1', '03_GBH_005', '01_HOOFD_